<a href="https://colab.research.google.com/github/andreaderuvo/rc_car_tracking/blob/main/C5_extract_close_frames_from_videos_for_deepsort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#config
PROJECT = 'rc_car_tracking'
MODEL = 'deepsort'

COLAB_BASE_PROJECT = f'/content/{PROJECT}'

DRIVE_MOUNT_FOLDER = 'drive'
DRIVE_BASE_PROJECT = f'{DRIVE_MOUNT_FOLDER}/MyDrive/{PROJECT}'
DRIVE_VIDEOS_FOLDER = 'videos'
DRIVE_IMAGES_FOLDER = f'{MODEL}_images'

# Interval of frames considered in percentages (0 -> 0%, 1->100%)
RANDOMNESS = False
MAX_NUMBER_OF_FRAMES_PER_VIDEO = 100
INTERVAL_FRAMES = (0.2,0.8)
SKIP_FRAMES = 40

In [ ]:
#imports
import cv2
import os, datetime
import random
from os import listdir
import shutil
import time
from google.colab import drive

In [ ]:
#google drive
drive.mount(DRIVE_MOUNT_FOLDER, force_remount=True)

In [ ]:
#symbolic link
!ln -s $DRIVE_BASE_PROJECT $COLAB_BASE_PROJECT

In [ ]:
# init DRIVE_IMAGES_FOLDER
shutil.rmtree(f'{COLAB_BASE_PROJECT}/{DRIVE_IMAGES_FOLDER}', ignore_errors=True)
os.makedirs(f'{COLAB_BASE_PROJECT}/{DRIVE_IMAGES_FOLDER}', exist_ok=True)

In [ ]:
# Extract random images in OUTPUT_IMAGES_FOLDER from each video in INPUT_VIDEOS_FOLDER
videos = os.listdir(f'{COLAB_BASE_PROJECT}/{DRIVE_VIDEOS_FOLDER}')

for i,video in enumerate(videos):
  
  if not video.endswith(".mp4"):
    continue;

  cap = cv2.VideoCapture(f'{COLAB_BASE_PROJECT}/{DRIVE_VIDEOS_FOLDER}/{video}')

  if not cap.isOpened(): 
    print(f'Could not open video {video}, {i+1}/{len(videos)}')
    break;

  length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  deck = list(range(int(length * INTERVAL_FRAMES[0]), int(length * INTERVAL_FRAMES[1]), SKIP_FRAMES))

  print(f'Processing video {video} with total {length} frames and {len(deck)} frames selected, {i+1}/{len(videos)}')

  if len(deck) < MAX_NUMBER_OF_FRAMES_PER_VIDEO:
    print(f'\nVideo {video} does not contain enough frames ({length} total frames, {len(deck)} selected frames) to extract in according to MAX_NUMBER_OF_FRAMES_PER_VIDEO ({MAX_NUMBER_OF_FRAMES_PER_VIDEO}) set')
    break;

  if RANDOMNESS:
    random.shuffle(deck)

  num_image = 0      
  counter = 0
  video_index = video.split('_')[0]

  while num_image < MAX_NUMBER_OF_FRAMES_PER_VIDEO:
    num_frame = deck.pop(0)
    cap.set(cv2.CAP_PROP_POS_FRAMES, num_frame - 1)
    success, image = cap.read()
    if success:
      file_name = f'{video_index}_{num_frame}.jpg'
      path = os.path.join(f'{COLAB_BASE_PROJECT}/{DRIVE_IMAGES_FOLDER}', file_name)

      cv2.imwrite(path, image)
      print(f"\rExtracting random frame {counter + 1} of {MAX_NUMBER_OF_FRAMES_PER_VIDEO}", end='', flush=True) 
      num_image += 1
      counter += 1
    else:
      print(f'Error in reading frame {num_frame}...')

  print('\n')